In [ ]:
import sys

import requests
import pandas as pd
from datetime import datetime
import boto3
from io import BytesIO

# Configurações
API_URL = "https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjIwLCJpbmRleCI6IklCT1YiLCJzZWdtZW50IjoiMSJ9"
S3_BUCKET = "bovespa-4176950"

def upload_raw_data():
    try:
        # 1. Coleta dos dados brutos
        response = requests.get(API_URL)
        response.raise_for_status()
        raw_data = response.json()
        
        # 2. Conversão direta para DataFrame (sem modificações)
        df = pd.DataFrame(raw_data['results'])  # Mantém TODOS os campos originais
        
        # 3. Gera o caminho de partição dinâmico (SEM adicionar colunas ao DataFrame)
        current_date = datetime.now()
        s3_partition_path = (
            f"raw/year={current_date.year}/"
            f"month={current_date.month:02d}/"
            f"day={current_date.day:02d}/"
            "dados_brutos.parquet"
        )
        
        # 4. Upload para S3
        s3 = boto3.client('s3')
        parquet_buffer = BytesIO()
        df.to_parquet(parquet_buffer, index=False)  # Arquivo Parquet puro
        
        s3.put_object(
            Bucket=S3_BUCKET,
            Key=s3_partition_path,
            Body=parquet_buffer.getvalue()
        )
        
        print(f"Dados brutos salvos em: s3://{S3_BUCKET}/{s3_partition_path}")
    
    except Exception as e:
        print(f"Erro: {e}")

# Execução
if __name__ == "__main__":
    upload_raw_data()

In [ ]:
import json
import requests
import pandas as pd
import boto3

from datetime import datetime
from io import BytesIO

def lambda_handler(event, context):
    API_URL = "https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/eyJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjIwLCJpbmRleCI6IklCT1YiLCJzZWdtZW50IjoiMSJ9"
    S3_BUCKET = "bovespa-4176950"

    try:
        response = requests.get(API_URL)
        response.raise_for_status()
        raw_data = response.json()

        df = pd.DataFrame(raw_data['results'])

        current_date = datetime.now()
        s3_partition_path = (
            f"raw/year={current_date.year}/"
            f"month={current_date.month:02d}/"
            f"day={current_date.day:02d}/"
            "dados_brutos.parquet"
        )

        s3 = boto3.client('s3')
        parquet_buffer = BytesIO()
        df.to_parquet(parquet_buffer, index=False)

        s3.put_object(
            Bucket=S3_BUCKET,
            Key=s3_partition_path,
            Body=parquet_buffer.getvalue()
        )

        return {
            'statusCode': 200,
            'body': f'Dados enviados para s3://{S3_BUCKET}/{s3_partition_path}'
        }

    except Exception as e:
        return {
            'statusCode': 500,
            'body': str(e)
        }

Response
{
  "errorMessage": "Unable to import module 'lambda_function': No module named 'requests'",
  "errorType": "Runtime.ImportModuleError",
  "requestId": "",
  "stackTrace": []
}